설명
- 각 코드의 작성자는 마크다운 셀의 가장 끝부분에 (이름)으로 작성함
- 만일 이름이 쓰여 있지 않은 경우, 바로 위와 작성자가 동일하거나, 대주제의 작성자와 동일함을 의미

# DBSCAN 모델링 (김정수)

## 필수 실행

In [ ]:
import pandas as pd
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors # eps 찾기: 최근접 이웃
from sklearn.metrics import silhouette_score  # 실루엣 스코어 함수
from sklearn.metrics import silhouette_samples

import folium
import random
from scipy.spatial import ConvexHull
from shapely.geometry import MultiPoint, Polygon
from shapely.ops import cascaded_union, polygonize
import matplotlib.pyplot as plt

# from sklearn.preprocessing import StandardScaler, LabelEncoder

from matplotlib.ticker import MultipleLocator # y축 눈금 단위 변경
import warnings;warnings.filterwarnings(action='ignore')

In [ ]:
df=pd.read_csv('REAL_LAST.csv')# 데이터 로딩
# df['TO_PORT_CD'].value_counts()# 도착항 종류 확인

# 각 도착 항구별 데이터프레임을 필터링한 사전 생성
port_dic={
    # 'SGSIN': df[df['TO_PORT_CD']=='SGSIN'],# 싱가폴항
    'HKHKG': df[df['TO_PORT_CD']=='HKHKG'],# 홍콩항
    # 'CNSHA': df[df['TO_PORT_CD']=='CNSHA'],# 상해항
    # 'KRPUS': df[df['TO_PORT_CD']=='KRPUS'],# 부산항
    # 'KRBNP': df[df['TO_PORT_CD']=='KRBNP'],# 부산신항
    # 'BUSAN': df[(df['TO_PORT_CD']=='KRPUS')|(df['TO_PORT_CD']=='KRBNP')],# 부산항 + 부산신항
    # 'KRINC': df[df['TO_PORT_CD']=='KRINC']# 인천항
}

## K-Distance Plot
- 항구별 min_pts에 따른 최적 eps 찾기

In [ ]:
def plot_k_distance(data,minpts):
    '''
    주어진 데이터에 대해 각 데이터 포인트의 k번째 최근접 이웃까지의 거리를 계산
    엘보우 포인트를 찾기 위해 Kneedle 알고리즘을 적용하여 이를 시각화

    Parameters:
    data: 데이터 포인트들이 포함된 배열 또는 데이터프레임(array-like)
    minpts: 각 데이터 포인트의 k번째 최근접 이웃을 지정하는 min_points 정수 값(int)

    Returns:
    elbow_value (float): 엘보우 포인트에서의 거리 값
    '''
    neighbors = NearestNeighbors(n_neighbors=minpts+1)# NearestNeighbors 객체 생성 min_samples + 1을 사용하는 이유: 각 데이터 포인트 자체를 제외하고 k개의 최근접 이웃을 고려하기 위함.
    neighbors_fit = neighbors.fit(data)# 데이터를 사용하여 이웃 관계를 학습. 각 데이터 포인트의 최근접 이웃을 계산함
    distances, indices = neighbors_fit.kneighbors(data)# 각 데이터 포인트의 k번째 최근접 이웃까지의 거리와 인덱스를 계산
    distances=np.sort(distances[:,minpts], axis=0)# 각 포인트의 minpts번째 최근접 이웃까지의 거리들을 오름차순으로 정렬

    n_points=len(distances)# 엘보우 포인트 찾기 (Kneedle 알고리즘 사용). 각 포인트로부터 직선까지의 거리를 계산하여 가장 큰 거리를 갖는 포인트를 엘보우 포인트로 정의함.
    all_coords=np.vstack((range(n_points), distances)).T# 모든 포인트의 좌표 생성
    line_vec=all_coords[-1]-all_coords[0]# 첫 번째와 마지막 포인트를 연결하는 직선 벡터를 계산
    line_vec_norm=line_vec / np.sqrt(np.sum(line_vec**2))# 직선 벡터를 정규화합니다.
    vec_from_first=all_coords - all_coords[0]# 첫 번째 포인트로부터 각 포인트까지의 벡터를 계산
    scalar_product=np.sum(vec_from_first*np.tile(line_vec_norm,(n_points, 1)),axis=1)# 각 포인트로부터 직선 벡터에 투영된 벡터의 스칼라 곱을 계산
    vec_from_first_parallel = np.outer(scalar_product,line_vec_norm)# 투영된 벡터를 계산
    vec_to_line=vec_from_first - vec_from_first_parallel# 직선으로부터 각 포인트까지의 벡터를 계산
    dist_to_line=np.sqrt(np.sum(vec_to_line**2,axis=1))# 직선으로부터 각 포인트까지의 거리를 계산
    elbow_point=np.argmax(dist_to_line)# 거리가 최대인 포인트 (엘보우 포인트) 찾기
    elbow_value=distances[elbow_point]

    # 그래프 생성
    fig, ax = plt.subplots(figsize=(8, 10))
    ax.plot(distances)  # 거리들을 플롯합니다.
    ax.scatter(elbow_point, elbow_value, color='red')  # 엘보우 포인트에 빨간 점을 그립니다.
    ax.set_xlabel('Points sorted by distance')
    ax.set_ylabel('Eps distance')
    ax.yaxis.set_label_position("right")# y축 눈금을 오른쪽으로 지정
    ax.yaxis.tick_right()# y축 눈금 표시
    ax.tick_params(axis='y', labelsize=7)# y축 폰트 사이즈를 설정
    ax.yaxis.set_major_locator(MultipleLocator(0.02))# y축 눈금을 0.002 단위로 설정
    ax.grid()# 그리드 추가
    plt.show()
    fig.savefig('plot_k_distance.png') # 이미지파일로 저장
    return elbow_value  # 엘보우 포인트의 거리 값 반환

minpts_list=[3,4,5,6]#,7,8,9,10]#,11,12,13,14,15,16,17,18,19,20]
for port_name,port_df in port_dic.items():# 
    data=port_df[['WAIT_LAT', 'WAIT_LON']].values
    for minpts in minpts_list:
        eps=plot_k_distance(data,minpts=minpts)
        print(f"{port_name} {minpts} {eps}")

In [ ]:
자동함수결과='''
SG|MS:3|EPS:0.08074060904526578
SG|MS:4|EPS:0.08635713056836447
SG|MS:5|EPS:0.09341279827197049
SG|MS:6|EPS:0.155255883064057
SG|MS:7|EPS:0.15966764062889582
SG|MS:8|EPS:0.175541090417602
SG|MS:9|EPS:0.19293032801766552
SG|MS:10|EPS:0.19439225172316935
SG|MS:11|EPS:0.19944888116255607
SG|MS:12|EPS:0.20652261321462803
SG|MS:13|EPS:0.22608745149830473
SG|MS:14|EPS:0.21348313214865614
SG|MS:15|EPS:0.21704518987989105
SG|MS:16|EPS:0.2250890885849423
SG|MS:17|EPS:0.226551073711867
SG|MS:18|EPS:0.23468415258171432
SG|MS:19|EPS:0.24208446275008919
SG|MS:20|EPS:0.2453046253824105

CN|MS:3|EPS:0.09992322084980912
CN|MS:4|EPS:0.13700932320101067
CN|MS:5|EPS:0.13737365998254739
CN|MS:6|EPS:0.13922684168291982
CN|MS:7|EPS:0.10250695322757387
CN|MS:8|EPS:0.12315898364715931
CN|MS:9|EPS:0.12502636922664406
CN|MS:10|EPS:0.1790388458435789
CN|MS:11|EPS:0.181559090601385
CN|MS:12|EPS:0.25654094974876707
CN|MS:13|EPS:0.2613223340818859
CN|MS:14|EPS:0.2660653080072643
CN|MS:15|EPS:0.2931185487153602
CN|MS:16|EPS:0.27130378195115756
CN|MS:17|EPS:0.30967397267448865
CN|MS:18|EPS:0.3120756576024496
CN|MS:19|EPS:0.3136900674264344
CN|MS:20|EPS:0.28182054639433235

PU|MS:3|EPS:0.0748418999825721
PU|MS:4|EPS:0.11002921730159017
PU|MS:5|EPS:0.12456606735786944
PU|MS:6|EPS:0.09310098930730021
PU|MS:7|EPS:0.10655369126407004
PU|MS:8|EPS:0.10853554579492151
PU|MS:9|EPS:0.10464778771669399
PU|MS:10|EPS:0.10634404843243132
PU|MS:11|EPS:0.11123495621880784
PU|MS:12|EPS:0.1351592971016048
PU|MS:13|EPS:0.1253377583372339
PU|MS:14|EPS:0.14275681945532473
PU|MS:15|EPS:0.14568669895705233
PU|MS:16|EPS:0.14630858033623365
PU|MS:17|EPS:0.14579520118645656
PU|MS:18|EPS:0.15175739245586778
PU|MS:19|EPS:0.15621674696715418
PU|MS:20|EPS:0.1593003630943798

BN|MS:3|EPS:0.1242575945727308
BN|MS:4|EPS:0.1429850677798234
BN|MS:5|EPS:0.14381732588600113
BN|MS:6|EPS:0.1609290891448752
BN|MS:7|EPS:0.16509158543363855
BN|MS:8|EPS:0.16460961622578082
BN|MS:9|EPS:0.17534835204529858
BN|MS:10|EPS:0.18798542392696357
BN|MS:11|EPS:0.1845098682293261
BN|MS:12|EPS:0.19087444196905917
BN|MS:13|EPS:0.22640702698679824
BN|MS:14|EPS:0.2355814763855376
BN|MS:15|EPS:0.23208235069689528
BN|MS:16|EPS:0.2653691845881927
BN|MS:17|EPS:0.2674057260437954
BN|MS:18|EPS:0.18531699503822993
BN|MS:19|EPS:0.19408296938421365
BN|MS:20|EPS:0.1955737098717601

IN|MS:3|EPS:0.06850623663579822
IN|MS:4|EPS:0.07185151390194464
IN|MS:5|EPS:0.08861061972473233
IN|MS:6|EPS:0.27646805517344625
IN|MS:7|EPS:0.33305825707942593
IN|MS:8|EPS:0.4072983165972852
IN|MS:9|EPS:0.5159673458075676
IN|MS:10|EPS:0.5903799627499691
IN|MS:11|EPS:0.7328756723623231
IN|MS:12|EPS:0.7434113589469006
IN|MS:13|EPS:0.7878343978850025

HK|MS:3|EPS:0.094733153573606
HK|MS:4|EPS:0.10743098022917223
HK|MS:5|EPS:0.11394079052298263
HK|MS:6|EPS:0.1263208844174252
HK|MS:7|EPS:0.13195806771850127
HK|MS:8|EPS:0.1433428158820613
HK|MS:9|EPS:0.14349849065756742
HK|MS:10|EPS:0.15044851645662594
HK|MS:11|EPS:0.15312628890559538
HK|MS:12|EPS:0.16784585347573855
HK|MS:13|EPS:0.2162801774227114
HK|MS:14|EPS:0.20308690483141048
HK|MS:15|EPS:0.20645032413876271
HK|MS:16|EPS:0.21173990512890228
HK|MS:17|EPS:0.22396494150871415
HK|MS:18|EPS:0.260120929194473
HK|MS:19|EPS:0.26791690470927704
HK|MS:20|EPS:0.27074180090263317

BUS|MS:3|EPS:0.09857033243323797
BUS|MS:4|EPS:0.09664671636947018
BUS|MS:5|EPS:0.12187417273973902
BUS|MS:6|EPS:0.1434845870642411
BUS|MS:7|EPS:0.14250598338315285
BUS|MS:8|EPS:0.16111214705912902
BUS|MS:9|EPS:0.1658745298621775
BUS|MS:10|EPS:0.10407860740805368
BUS|MS:11|EPS:0.1123289941066125
BUS|MS:12|EPS:0.1123289941066125
BUS|MS:13|EPS:0.11501322924343271
BUS|MS:14|EPS:0.11912070869500636
BUS|MS:15|EPS:0.12105725834083343
BUS|MS:16|EPS:0.12130433561912299
BUS|MS:17|EPS:0.12093823865923732
BUS|MS:18|EPS:0.12223902536015237
BUS|MS:19|EPS:0.11547525682154626
BUS|MS:20|EPS:0.1162731881819802
'''

In [ ]:
# 직전 코드의 플롯 생성 결과를 보고 항구별 min_points에 따른 최적 epsilon 값 찾기
eps_by_eyes='''KRPUS 15 0.1457
SGSIN 9 0.151
KRBNP 7 0.1651
HKHKG 19 0.374
CNSHA 6 0.1392
KRINC 8 0.38
'''

# '''
# SGSIN 3 0.0807, 0.0449, 0.037, 0.021, 0.013
# SGSIN 4 0.0864, 0.07, 0.049, 0.026
# SGSIN 5 0.0934, 0.077, 0.059
# SGSIN 6 0.1553, 0.1, 0.089, 0.037
# SGSIN 7 0.1597, 0.115, 0.103, 0.045, 0.023
# SGSIN 8 0.1755, 0.126, 0.092, 0.084
# SGSIN 9 0.1929, 0.151, 0.096
# SGSIN 10 0.1944
# HKHKG 3 0.1992, 0.1977, 0.1367, 0.132, 0.0947, 0.083, 0.077, 0.068, 0.062
# HKHKG 4 0.2272, 0.2163, 0.1074, 0.0925, 0.0855, 0.08
# HKHKG 5 0.235, 0.219, 0.2063, 0.166, 0.1617, 0.1139, 0.1, 0.0976, 0.085, 0.08, 0.0476, 0.036
# HKHKG 6 0.3192, 0.318, 0.2447, 0.206, 0.1723, 0.1263, 0.117, 0.108, 0.099, 0.086, 0.071, 0.0635, 0.051, 0.028
# HKHKG 7 0.324, 0.318, 0.2462, 0.259, 0.2221, 0.204, 0.1723, 0.132, 0.1, 0.0875, 0.071, 0.062, 0.0375, 0.028
# HKHKG 8 0.3357, 0.251, 0.248, 0.245, 0.216, 0.282, 0.271, 0.232, 0.228, 0.208, 0.1433, 0.1225, 0.113, 0.1025, 0.07, 0.042, 0.033
# HKHKG 9 0.1435, 0.13, 0.117, 0.097, 0.0776, 0.07, 0.0525
# HKHKG 10 0.294, 0.2823, 0.2322, 0.232, 0.1504, 0.1275, 0.091, 0.055
# HKHKG 11 0.298, 0.288, 0.2718, 0.268, 0.2222
# HKHKG 12 0.342, 0.3333, 0.302, 0.2941, 0.270, 0.242
# HKHKG 13 0.3463, 0.332, 0.303, 0.292, 0.294, 0.244
# HKHKG 14 0.3492, 0.3399, 0.3381, 0.306, 0.2498, 0.226
# HKHKG 15 0.3559, 0.350, 0.344, 0.3123, 0.307, 0.252, 0.234, 0.20645032413876271
# HKHKG 16 0.3682, 0.354, 0.3157, 0.3092, 0.284, 0.2522, 0.238, 0.21173990512890228
# HKHKG 17 0.22396494150871415, 0.37, 0.325, 0.293, 0.256, 0.192, 0.1727, 0.12, 0.087, 0.078, 0.0648
# HKHKG 18 0.260120929194473, 0.378, 0.326, 0.246, 0.122, 0.093, 0.082
# HKHKG 19 0.26791690470927704, 0.4175, 0.374, 0.1642, 0.1235, 0.0833
# HKHKG 20 0.27074180090263317, 0.592, 0.424, 0.376, 0.316, 0.304, 0.255, 0.2175, 0.2005, 0.143, 0.082
# KRPUS 3 0.0748, 0.0715, 0.0625, 0.058, 0.05375, 0.049, 0.042, 0.035, 0.0265
# KRPUS 4 0.11, 0.08, 0.065, 0.0575, 0.046
# KRPUS 5 0.1246, 0.115, 0.103, 0.0938, 0.087, 0.0675
# KRPUS 6 0.0931, 0.0865, 0.0685, 0.047
# KRPUS 7 0.1066, 0.098, 0.0815
# KRPUS 8 0.1085, 0.09, 0.085, 0.065, 0.045, 0.036
# KRPUS 9 0.1046, 0.093, 0.0865, 0.067, 0.0465
# KRPUS 10 0.1063, 0.072, 0.0485
# KRPUS 11 0.1112, 0.073, 0.0455
# KRPUS 12 0.1352, 0.094, 0.0835, 0.0765
# KRPUS 13 0.1253, 0.114, 0.107, 0.101, 0.077
# KRPUS 14 0.1428, 0.1077, 0.0885
# KRPUS 15 0.1457, 0.1275,, 0.0975, 0.082
# KRBNP 3 0.1243, 0.1045, 0.095, 0.088, 0.0775, 0.0045
# KRBNP 4 0.143, 0.1225, 0.116, 0.097, 0.0875, 0.0785, 0.0076
# KRBNP 5 0.1438, 0.135, 0.125, 0.082, 0.043, 0.0077
# KRBNP 6 0.1609, 0.1435, 0.134, 0.1135, 0.057, 0.012
# KRBNP 7 0.1651, 0.138, 0.118, 0.088, 0.0155, 0.0125
# KRBNP 8 0.1646, 0.1175, 0.085, 0.015, 0.013
# KRBNP 9 0.1753, 0.119, 0.1035, 0.079, 0.013
# KRBNP 10 0.188, 0.1645, 0.149, 0.1435, 0.122, 0.115, 0.0175
# KRBNP 11 0.1845, 0.155, 0.1325, 0.0225
# KRBNP 12 0.1909, 0.165, 0.157, 0.022
# KRBNP 13 0.2264, 0.1965, 0.1045, 0.024
# KRBNP 14 0.2356, 0.2075, 0.105, 0.024
# KRBNP 15 0.2321, 0.219, 0.2095, 0.1875, 0.161, 0.11, 0.025
# CNSHA 3 0.0999, 0.065
# CNSHA 4 0.137, 0.110, 0.095, 0.077
# CNSHA 5 0.1374, 0.112, 0.1, 0.08, 0.062, 0.0515, 0.043
# CNSHA 6 0.1392, 0.134, 0.099, 0.08, 0.0725, 0.0515
# CNSHA 7 0.1025, 0.082, 0.072, 0.056
# CNSHA 8 0.1232, 0.114, 0.102, 0.083, 0.074, 0.06
# CNSHA 9 0.125, 0.116, 0.1085, 0.1, 0.083, 0.06
# CNSHA 10 0.179, 0.133, 0.085, 0.06, 0.05
# BUSAN 3 0.0986, 0.09, 0.0855, 0.07875, 0.07, 0.0435, 0.003
# BUSAN 4 0.0966, 0.087, 0.057
# BUSAN 5 0.1219, 0.11, 0.062
# BUSAN 6 0.1435, 0.12375, 0.0875, 0.076
# BUSAN 7 0.1425, 0.0935, 0.085, 0.0675
# BUSAN 8 0.1611, 0.138, 0.1185, 0.0995, 0.09, 0.0838, 0.065
# BUSAN 9 0.1659, 0.1538, 0.1465, 0.1435, 0.0935, 0.067, 0.052
# BUSAN 10 0.1041, 0.0945, 0.0835, 0.075, 0.0285, 0.018, 0.015
# BUSAN 11 0.1123, 0.0935, 0.087, 0.0155
# BUSAN 12 0.1123, 0.0965, 0.0803
# BUSAN 13 0.115, 0.1095, 0.10195, 0.09595, 0.0648, 0.022
# BUSAN 14 0.1191, 0.101, 0.0975, 0.014
# BUSAN 15 0.1211, 0.0995, 0.081, 0.0695, 0.015
# KRINC 3 0.0685, 0.0418, 0.0188
# KRINC 4 0.3215, 0.07185, 0.043
# KRINC 5 0.386, 0.08861, 0.072
# KRINC 6 0.248, 0.076468, 0.02085
# KRINC 7 0.4375, 0.333, 0.2728
# KRINC 8 0.4525, 0.40729, 0.38, 0.344
# KRINC 9 0.51596, 0.491, 0.4585
# '''

# 항구별 min_points에 따른 최적 epsilon 값을 이중리스트에 담기
port_minpts_eps=[]

for line in eps_by_eyes.strip().split('\n'):
    parts=line.split()
    port=parts[0]
    minpts=int(parts[1])
    eps_values=[float(eps.strip(',')) for eps in parts[2:]]
    for eps in eps_values:
        port_minpts_eps.append([port, minpts, eps])

# port_minpts_eps

## DBSCAN
<pre>
- DBSCAN 알고리즘: 데이터 포인트 간의 밀집도를 기반으로 군집을 형성하는 비지도 학습 방법

1. 각 항구의 데이터를 DBSCAN 알고리즘을 사용해 군집화
2. Folium 맵을 사용해 시각화
3. 각 항구에 대해 여러 가지 eps와 min_samples 조합을 적용하여 군집화하고, 군집 결과를 Folium 맵에 시각화
4. Convex Hull을 사용해 군집의 경계를 시각적으로 표시
5. 각 군집의 위치와 범위를 쉽게 파악 가능
6. 최종 결과 맵은 HTML 파일로 저장
</pre>

In [ ]:
df=pd.read_csv('REAL_LAST.csv')

# 각 항구별 데이터프레임을 필터링한 사전 생성
port_dic={
    'SGSIN': df[df['TO_PORT_CD']=='SGSIN'],# 상해항
    'HKHKG': df[df['TO_PORT_CD']=='HKHKG'],# 홍콩항
    'KRINC': df[df['TO_PORT_CD']=='KRINC'],# 인천항
    'KRPUS': df[df['TO_PORT_CD']=='KRPUS'],# 부산항
    'KRBNP': df[df['TO_PORT_CD']=='KRBNP'],# 부산신항
    'CNSHA': df[df['TO_PORT_CD']=='CNSHA'],# 상해항
    # 'BUSAN': df[(df['TO_PORT_CD']=='KRPUS')|(df['TO_PORT_CD']=='KRBNP')] # 부산항 + 부산신항
}

# Convex Hull을 생성하는 함수
def create_convex_hull(cluster_points):
    """
    클러스터에 속한 점들을 감싸는 최소한의 다각형(Convex Hull)을 생성하는 함수.
    클러스터에 속한 점이 3개 미만일 경우 None을 반환.
    """
    if len(cluster_points)<3:
        return None
    points=MultiPoint(cluster_points)  # 점들을 MultiPoint 객체로 생성
    hull=points.convex_hull  # Convex Hull 생성
    return hull

# Folium 맵에 군집 경계를 추가하는 함수
def add_cluster_boundaries(map_obj,data,labels,cluster_colors):
    """
    Folium 맵에 군집 경계를 추가하는 함수.
    군집의 Convex Hull을 생성하고, 이를 맵에 폴리곤 형태로 추가.

    Parameters:
    - map_obj: Folium 맵 객체
    - data: 군집화할 데이터 (위도, 경도)
    - labels: 군집 라벨
    - cluster_colors: 각 군집의 색상 딕셔너리
    """
    unique_labels = set(labels)  # 유니크한 군집 라벨들
    for label in unique_labels:
        if label == -1:
            continue  # 노이즈 점은 무시
        cluster_points=data[labels == label][:, [1, 0]]  # 군집에 속한 점들의 경도, 위도 추출
        hull=create_convex_hull(cluster_points)  # 군집의 Convex Hull 생성
        if hull and isinstance(hull,Polygon):  # Convex Hull이 유효한 경우
            folium.Polygon(
                locations=[(point[1],point[0]) for point in hull.exterior.coords],  # Convex Hull의 외곽 좌표
                color=cluster_colors[label],  # 군집 색상
                weight=2,
                fill=True,
                fill_color=cluster_colors[label],
                fill_opacity=0.3,
                tooltip=f"Cluster: {label}"  # 군집 번호 표시
            ).add_to(map_obj)

# 결과 파일 저장 경로
save_path="/Users/bara/Library/CloudStorage/GoogleDrive-kitadima45@gmail.com/내 드라이브/김정수/0617시각화"

# 각 항구별로 DBSCAN 군집화 수행 및 Folium 맵 생성
for port_name,port_df in port_dic.items():
    """
    각 항구별로 데이터를 필터링하여 DBSCAN 군집화 알고리즘을 적용하고, Folium 맵을 생성하여 군집 결과를 시각화.
    Parameters:
    - port_name: 항구 이름
    - port_data: 해당 항구의 데이터프레임
    """
    data=port_df[['WAIT_LAT', 'WAIT_LON']].values  # 대기 지점의 위도, 경도 추출
    for minpts, eps in [(item[1], item[2]) for item in port_minpts_eps if item[0]==port_name]:
        """
        각 항구에 대해 (min points, epsilon)을 조합하여 DBSCAN 군집화 알고리즘을 여러 번 수행.
        """
        # DBSCAN 군집화 수행
        labels = DBSCAN(eps=eps, min_samples=minpts).fit_predict(data)

        # 실루엣 샘플 평가 값 추가
        if len(set(labels)) > 1:
            sil_samples = silhouette_samples(data, labels)  # 실루엣 계수 계산
        else:
            sil_samples = [-1] * len(labels)  # 군집이 하나인 경우 실루엣 계수 계산 불가

        # Folium 맵 생성
        map_port = folium.Map(location=[data[:, 0].mean(), data[:, 1].mean()], zoom_start=8.5)  # 맵 초기화
        unique_labels = set(labels)  # 유니크한 군집 라벨들
        cluster_colors = {label: (f'#{np.random.randint(0, 0xFFFFFF):06x}' if label != -1 else '#000000') for label in unique_labels}  # 군집 색상 설정

        for idx, row in enumerate(data):
            """
            각 점을 Folium CircleMarker로 표시.
            """
            marker_radius = 0.8 if labels[idx] == -1 else 2.5  # 노이즈 점은 작게, 군집화된 점은 크게 표시
            marker_color = '#000000' if labels[idx] == -1 else cluster_colors.get(labels[idx], '#000000')  # 노이즈 점은 검정색으로 표시
            folium.CircleMarker(
                location=[row[0], row[1]], # 선박별 대기 지점의 위도, 경도 표시
                radius=marker_radius,
                color=marker_color,
                fill=True,
                fill_color=marker_color,
                fill_opacity=1.5,
            ).add_to(map_port)

        # 군집 경계 추가
        add_cluster_boundaries(map_port, data, labels, cluster_colors)

        # 파일명 동적으로 생성 및 저장
        map_port.save(f"{save_path}/{port_name}_{minpts}_{eps}.html")  # 결과 맵을 HTML 파일로 저장

print('END')

### 항구별 csv 저장(군집번호 컬럼 추가)

In [ ]:
import pandas as pd
from sklearn.cluster import DBSCAN

# 데이터 불러오기
df = pd.read_csv('REAL_LAST.csv')

# [도착 항구 이름, min_points, eps]
port_minpts_eps=[
    ['KRPUS', 15, 0.1457],
    ['SGSIN', 9, 0.151],
    ['KRBNP', 7, 0.1651],
    ['HKHKG', 19, 0.374],
    ['CNSHA', 6, 0.1392],
    ['KRINC', 8, 0.38]
]

# 각 도착 항구별 데이터프레임 생성
port_dic = {
    'SGSIN': df[df['TO_PORT_CD'] == 'SGSIN'],
    'HKHKG': df[df['TO_PORT_CD'] == 'HKHKG'],
    'KRINC': df[df['TO_PORT_CD'] == 'KRINC'],
    'KRPUS': df[df['TO_PORT_CD'] == 'KRPUS'],
    'KRBNP': df[df['TO_PORT_CD'] == 'KRBNP'],
    'CNSHA': df[df['TO_PORT_CD'] == 'CNSHA']
}

# 각 항구별로 DBSCAN 군집화 수행 및 결과 저장
for port_name, port_df in port_dic.items():
    # 대기 지점의 위도, 경도 추출
    data = port_df[['WAIT_LAT', 'WAIT_LON']].values

    # 해당 항구의 minpts와 eps 값 가져오기
    for minpts, eps in [(item[1], item[2]) for item in port_minpts_eps if item[0]==port_name]:
        # DBSCAN 군집화 수행
        labels = DBSCAN(eps=eps, min_samples=minpts).fit_predict(data)

        # 'Cluster_Label' 컬럼 추가
        port_df['Cluster_Label'] = labels

        # 결과를 CSV 파일로 저장
        output_filename = f"REAL_LAST_{port_name}.csv"
        port_df.to_csv(output_filename, index=False)

print('END')

### (번외) 군집화 되지 않은 이상치를 제거한 파일 생성
- Cluster_Label=-1 제거

In [ ]:
df_CNSHA=pd.read_csv('REAL_LAST_CNSHA.csv')
df_HKHKG=pd.read_csv('REAL_LAST_HKHKG.csv')
df_KRBNP=pd.read_csv('REAL_LAST_KRBNP.csv')
df_SGSIN=pd.read_csv('REAL_LAST_SGSIN.csv')
df_KRPUS=pd.read_csv('REAL_LAST_KRPUS.csv')
df_KRINC=pd.read_csv('REAL_LAST_KRINC.csv')

port_name_dic={
    'SGSIN': df_SGSIN,
    'HKHKG': df_HKHKG,
    'CNSHA': df_CNSHA,
    'KRPUS': df_KRPUS,
    'KRBNP': df_KRBNP,
    'KRINC': df_KRINC
}

# for port_name, port_df in port_name_dic.items():
#     if port_name!='SGSIN':
        # if port_df['Cluster_Label']!=-1:
        #         port_df['Cluster_Label']=port_name
        # else:
        #     if port_df['Cluster_Label']!=-1:
        #         if port_df['Cluster_Label']==0:
        #             port_df['Cluster_Label']='SGSIN_0'
        #         elif port_df['Cluster_Label']==1:
        #             port_df['Cluster_Label']='SGSIN_0'
        #         elif port_df['Cluster_Label']==2:
        #             port_df['Cluster_Label']='SGSIN_0'

for port_name, port_df in port_name_dic.items():
    if port_name!='SGSIN':
        port_df.loc[port_df['Cluster_Label']!= -1,'Cluster_Label']=port_name
    else:
        port_df['Cluster_Label']="SGSIN_"+port_df['Cluster_Label'].astype(str)
THE_LAST_df = pd.concat(port_name_dic.values(), ignore_index=True)
THE_LAST_df.to_csv('THE_LAST_df.csv',index=False)

In [ ]:
df_HKHKG['Cluster_Label'].value_counts()
display(df_HKHKG[df_HKHKG['WAIT_LON']==113.3843])

## DBSCAN 결과 시각화

In [ ]:
# DBSCAN: 밀도 기반 군집화 함수 생성
def apply_dbscan(data, eps, min_samples):
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(data)
    labels = db.labels_
    return labels

# Convex Hull을 생성하여 군집 경계를 표시하는 함수
def create_convex_hull(cluster_points):
    if len(cluster_points) < 3:
        return None
    points = MultiPoint(cluster_points)
    hull = points.convex_hull
    return hull

# Folium 맵에 군집 경계를 추가하는 함수
def add_cluster_boundaries(map_obj, data, labels, cluster_colors):
    unique_labels = set(labels)
    for label in unique_labels:
        if label == -1:
            continue
        cluster_points = data[labels == label][:, [1, 0]]  # Reverse order for lat, lon
        hull = create_convex_hull(cluster_points)
        if hull and isinstance(hull, Polygon):
            folium.Polygon(
                locations=[(point[1], point[0]) for point in hull.exterior.coords],
                color=cluster_colors[label],
                weight=2,
                fill=True,
                fill_color=cluster_colors[label],
                fill_opacity=0.3
            ).add_to(map_obj)

# 항구별로 군집화 및 시각화
def process_port(data, eps, min_samples, port_name):
    data = data.dropna(subset=['WAIT_LAT', 'WAIT_LON'])
    coords = data[['WAIT_LAT', 'WAIT_LON']].values
    labels = apply_dbscan(coords, eps, min_samples)
    data['cluster'] = labels

    unique_labels = set(labels)
    cluster_colors = {label: (f'#{np.random.randint(0, 0xFFFFFF):06x}' if label != -1 else '#000000') for label in unique_labels}

    # Folium 맵 생성
    map_port = folium.Map(location=[22.5,116],zoom_start=4)

    # 기존 마커 추가
    for idx, row in data.iterrows():
        if pd.isna(row['cluster']):
            continue
        marker_radius = 1 if row['cluster'] == -1 else 2  # 노이즈 값의 마커 크기 조정
        folium.CircleMarker(
            location=[row['WAIT_LAT'], row['WAIT_LON']],
            radius=marker_radius,
            color=cluster_colors.get(row['cluster'], '#000000'),
            fill=True,
            fill_color=cluster_colors.get(row['cluster'], '#000000'),
            fill_opacity=1.5,
            popup=f"ID: {row['VSL_ID']}<br>Port: {port_name}<br>항해:{row['TRIP']}<br>Cluster: {row['cluster']}<br>대기시간: {row['WAITING_TIME']}"
        ).add_to(map_port)
    # 군집 경계 추가
    add_cluster_boundaries(map_port, coords, labels, cluster_colors)
    return map_port, data, cluster_colors

# 각 항구 데이터
ports = {
    '싱가포르':{'data':df[df['TO_PORT_CD']=='SGSIN'], 'eps': 0.09, 'min_samples': 4},
    '부산신항':{'data':df[df['TO_PORT_CD']=='KRBNP'], 'eps': 0.03, 'min_samples': 7},
    '부산항':{'data':df[df['TO_PORT_CD']=='KRBNP'],'eps': 0.03, 'min_samples': 7},
    '인천':{'data':df[df['TO_PORT_CD']=='KRINC'], 'eps': 0.05, 'min_samples': 4},
    '상해':{'data':df[df['TO_PORT_CD']=='CNSHA'], 'eps': 0.035, 'min_samples': 6},
    '홍콩':{'data': df[df['TO_PORT_CD']=='HKHKG'], 'eps': 0.0146, 'min_samples': 7}
}

# 모든 항구의 데이터 통합 및 시각화
all_ports_combined = pd.DataFrame()
map_all_ports = folium.Map(location=[0, 0], zoom_start=2)

for port_name, port_info in ports.items():
    map_port, processed_data, cluster_colors = process_port(port_info['data'], port_info['eps'], port_info['min_samples'], port_name)
    all_ports_combined = pd.concat([all_ports_combined, processed_data], ignore_index=True)

    # 통합 맵에 마커 추가
    for idx, row in processed_data.iterrows():
        if pd.isna(row['cluster']):
            continue
        marker_radius = 1 if row['cluster'] == -1 else 2  # 노이즈 값의 마커 크기 조정
        folium.CircleMarker(
            location=[row['WAIT_LAT'], row['WAIT_LON']],
            radius=marker_radius,
            color=cluster_colors.get(row['cluster'], '#000000'),
            fill=True,
            fill_color=cluster_colors.get(row['cluster'], '#000000'),
            fill_opacity=1.5,
            popup=f"ID: {row['VSL_ID']}<br>Port: {port_name}<br>Cluster: {row['cluster']}<br>TIME: {row['VSL_TIMESTAMP']}"
        ).add_to(map_all_ports)
    
    # 통합 맵에 군집 경계 추가
    coords = processed_data[['WAIT_LAT', 'WAIT_LON']].values
    labels = processed_data['cluster'].values
    add_cluster_boundaries(map_all_ports, coords, labels, cluster_colors)

# 전체 항구 데이터 시각화
map_all_ports.fit_bounds(map_all_ports.get_bounds())
map_all_ports.save("C.html")

print('END')

## 항구별 지도 저장

In [ ]:
import folium
from sklearn.cluster import DBSCAN
import pandas as pd
import numpy as np
from shapely.geometry import MultiPoint, Polygon

def apply_dbscan(data, eps, min_samples):
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(data)
    labels = db.labels_
    return labels

def create_convex_hull(cluster_points):
    if len(cluster_points) < 3:
        return None
    points = MultiPoint(cluster_points)
    hull = points.convex_hull
    return hull

def add_cluster_boundaries(map_obj, data, labels, cluster_colors):
    unique_labels = set(labels)
    for label in unique_labels:
        if label == -1:
            continue
        cluster_points = data[labels == label][:, [1, 0]] # 경도, 위도 순으로 변경
        hull = create_convex_hull(cluster_points)
        if hull and isinstance(hull, Polygon):
            folium.Polygon(
                locations=[(point[1], point[0]) for point in hull.exterior.coords],
                color=cluster_colors[label],
                weight=2,
                fill=True,
                fill_color=cluster_colors[label],
                fill_opacity=0.3,
                tooltip=f"Cluster: {label}"
            ).add_to(map_obj)

def process_port(data, eps, min_samples, port_name):
    data = data.dropna(subset=['WAIT_LAT', 'WAIT_LON'])
    coords = data[['WAIT_LAT', 'WAIT_LON']].values
    labels = apply_dbscan(coords, eps, min_samples)
    data['cluster'] = labels
    
    unique_labels = set(labels)
    cluster_colors = {label: (f'#{np.random.randint(0, 0xFFFFFF):06x}' if label != -1 else '#000000') for label in unique_labels}
    
    map_port = folium.Map(location=[22.5, 116], zoom_start=4)
    
    for idx, row in data.iterrows():
        if pd.isna(row['cluster']):
            continue
        marker_radius = 1 if row['cluster'] == -1 else 2
        folium.CircleMarker(
            location=[row['WAIT_LAT'], row['WAIT_LON']],
            radius=marker_radius,
            color=cluster_colors.get(row['cluster'], '#000000'),
            fill=True,
            fill_color=cluster_colors.get(row['cluster'], '#000000'),
            fill_opacity=1.5,
            popup=folium.Popup(f"ID: {row['VSL_ID']}<br>Port: {port_name}<br>항해: {row['TRIP']}<br>Cluster: {row['cluster']}<br>대기시간: {row['WAITING_TIME']}", max_width=300)
        ).add_to(map_port)
    
    add_cluster_boundaries(map_port, coords, labels, cluster_colors)
    
    return map_port, data, cluster_colors

ports = {
    '싱가포르': {'data': df[df['TO_PORT_CD'] == 'SGSIN'], 'eps': 0.09, 'min_samples': 4},
    '부산신항': {'data': df[df['TO_PORT_CD'] == 'KRBNP'], 'eps': 0.03, 'min_samples': 7},
    '부산항': {'data': df[df['TO_PORT_CD'] == 'KRBNP'], 'eps': 0.03, 'min_samples': 7},
    '인천': {'data': df[df['TO_PORT_CD'] == 'KRINC'], 'eps': 0.05, 'min_samples': 4},
    '상해': {'data': df[df['TO_PORT_CD'] == 'CNSHA'], 'eps': 0.035, 'min_samples': 6},
    '홍콩': {'data': df[df['TO_PORT_CD'] == 'HKHKG'], 'eps': 0.0146, 'min_samples': 7}
}

for port_name, port_info in ports.items():
    map_port, processed_data, cluster_colors = process_port(port_info['data'], port_info['eps'], port_info['min_samples'], port_name)
    map_port.fit_bounds(map_port.get_bounds())
    map_port.save(f"{port_name}.html")

print('END')

## (번외) 군집화하지 않은 선박 시각화

In [ ]:
df_grouped=df.groupby('VSL_ID')

m = folium.Map(location=[22.5,116],zoom_start=4)

# 선박 속도에 따라 색상을 생성하는 함수 (예시 함수, 필요시 실제 로직으로 대체)
def color_producer(id):
    return "#{:06x}".format(random.randint(0, 0xFFFFFF))

# 선박 위치를 마킹
for vsl_id, group in df_grouped:
    for idx, row in group.iterrows():
        folium.CircleMarker(
            location=[row['WAIT_LAT'], row['WAIT_LON']],
            radius=1,
            color=color_producer(row['VSL_ID']),
            fill=True,
            fill_color=color_producer(row['TO_PORT_CD']),
            fill_opacity=1,
            tooltip=f"선박: {row['VSL_ID']}<br>항해: {row['TRIP']}<br>대기시간: {row['WAITING_TIME']}<br>위도: {row['WAIT_LAT']}<br>경도: {row['WAIT_LON']}"
        ).add_to(m)

m.save('RawMap_CENTERS.html')

# 실루엣 계수, 실루엣 샘플 수 파악 (김묘경)
- 모델링 코드 및 min sample, eps은 김정수님의 코드를 참고및 사용하였습니다.

## 필수 실행 

## 모델링용 정보 처리 
- 군집화용 컬럼 생성
- 목적항 별로 분할

## 1개 항구 1개 모델에 대한 선박 면적 정보 시각화 

In [ ]:
df_port = df_SGSIN

In [ ]:
from datetime import timedelta
def str_to_timedelta(time_str):
    days, time = time_str.split(' days ')
    hours, minutes, seconds = map(int, time.split(':'))
    return timedelta(days=int(days), hours=hours, minutes=minutes, seconds=seconds)
df_port['WAITING_TIME(H:M:S)'] = df_port['WAITING_TIME(H:M:S)'].apply(str_to_timedelta)

In [ ]:
df_port['WAITING_TIME(H:M:S)'] = df_port['WAITING_TIME(H:M:S)'].apply(lambda x: x.total_seconds())

In [ ]:
df_port.groupby('cluster')[['WAITING_TIME(H:M:S)','GROSS_TONNAGE']].agg(['mean','std','max','min'])

In [ ]:
sns.set_style('whitegrid')
sns.boxplot(data=df_port, x='cluster',y='SQUARE')
plt.show()

In [ ]:
sns.violinplot(data=df_port,x='cluster',y='SQUARE')
plt.show()

In [ ]:
sns.kdeplot(data=df['GROSS_TONNAGE'], fill=True, ec='gray',fc='white')
sns.kdeplot(data=df_port[df_port['cluster']==0]['GROSS_TONNAGE'],fill=True,label='0')
sns.kdeplot(data=df_port[df_port['cluster']==1]['GROSS_TONNAGE'],fill=True,label='1')
sns.kdeplot(data=df_port[df_port['cluster']==2]['GROSS_TONNAGE'],fill=True,label='2')
sns.kdeplot(data=df_port[df_port['cluster']==-1]['GROSS_TONNAGE'],fill=True,label='-1')
plt.legend() 

## 모델링된 데이터 평가 점수 그래프 그리기

In [ ]:
import pandas as pd
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors # eps 찾기: 최근접 이웃
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.cluster import OPTICS
from sklearn.metrics import silhouette_score  # 실루엣 스코어 함수 임포트
from sklearn.metrics import silhouette_samples

import folium
import random
from scipy.spatial import ConvexHull
from shapely.geometry import MultiPoint, Polygon
from shapely.ops import cascaded_union, polygonize
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, LabelEncoder

from matplotlib.ticker import MultipleLocator # y축 눈금 단위 변경
import warnings;warnings.filterwarnings(action='ignore')

In [ ]:
df=pd.read_csv('REAL_LAST.csv')

In [ ]:
eps_by_eyes= """
SGSIN 3 0.0807, 0.0449, 0.037, 0.021, 0.013
SGSIN 4 0.0864, 0.07, 0.049, 0.026
SGSIN 5 0.0934, 0.077, 0.059
SGSIN 6 0.1553, 0.1, 0.089, 0.037
SGSIN 7 0.1597, 0.115, 0.103, 0.045, 0.023
SGSIN 8 0.1755, 0.126, 0.092, 0.084
SGSIN 9 0.1929, 0.151, 0.096
SGSIN 10 0.1944
HKHKG 3 0.0947, 0.083, 0.077, 0.068, 0.062
HKHKG 4 0.1074, 0.0925, 0.0855, 0.08
HKHKG 5 0.1139, 0.1, 0.0976, 0.085, 0.08, 0.0476, 0.036
HKHKG 6 0.1263, 0.117, 0.108, 0.099, 0.086, 0.071, 0.0635, 0.051, 0.028
HKHKG 7 0.132, 0.1, 0.0875, 0.071, 0.062, 0.0375, 0.028
HKHKG 8 0.1433, 0.1225, 0.113, 0.1025, 0.07, 0.042, 0.033
HKHKG 9 0.1435, 0.13, 0.117, 0.097, 0.0776, 0.07, 0.0525
HKHKG 10 0.1504, 0.1275, 0.091, 0.055
KRPUS 3 0.0748, 0.0715, 0.0625, 0.058, 0.05375, 0.049, 0.042, 0.035, 0.0265
KRPUS 4 0.11, 0.08, 0.065, 0.0575, 0.046
KRPUS 5 0.1246, 0.115, 0.103, 0.0938, 0.087, 0.0675
KRPUS 6 0.0931, 0.0865, 0.0685, 0.047
KRPUS 7 0.1066, 0.098, 0.0815
KRPUS 8 0.1085, 0.09, 0.085, 0.065, 0.045, 0.036
KRPUS 9 0.1046, 0.093, 0.0865, 0.067, 0.0465
KRPUS 10 0.1063, 0.072, 0.0485
KRPUS 11 0.1112, 0.073, 0.0455
KRPUS 12 0.1352, 0.094, 0.0835, 0.0765
KRPUS 13 0.1253, 0.114, 0.107, 0.101, 0.077
KRPUS 14 0.1428, 0.1077, 0.0885
KRPUS 15 0.1457, 0.1275,, 0.0975, 0.082
KRBNP 3 0.1243, 0.1045, 0.095, 0.088, 0.0775, 0.0045
KRBNP 4 0.143, 0.1225, 0.116, 0.097, 0.0875, 0.0785, 0.0076
KRBNP 5 0.1438, 0.135, 0.125, 0.082, 0.043, 0.0077
KRBNP 6 0.1609, 0.1435, 0.134, 0.1135, 0.057, 0.012
KRBNP 7 0.1651, 0.138, 0.118, 0.088, 0.0155, 0.0125
KRBNP 8 0.1646, 0.1175, 0.085, 0.015, 0.013
KRBNP 9 0.1753, 0.119, 0.1035, 0.079, 0.013
KRBNP 10 0.188, 0.1645, 0.149, 0.1435, 0.122, 0.115, 0.0175
KRBNP 11 0.1845, 0.155, 0.1325, 0.0225
KRBNP 12 0.1909, 0.165, 0.157, 0.022
KRBNP 13 0.2264, 0.1965, 0.1045, 0.024
KRBNP 14 0.2356, 0.2075, 0.105, 0.024
KRBNP 15 0.2321, 0.219, 0.2095, 0.1875, 0.161, 0.11, 0.025
CNSHA 3 0.0999, 0.065
CNSHA 4 0.137, 0.110, 0.095, 0.077
CNSHA 5 0.1374, 0.112, 0.1, 0.08, 0.062, 0.0515, 0.043
CNSHA 6 0.1392, 0.134, 0.099, 0.08, 0.0725, 0.0515
CNSHA 7 0.1025, 0.082, 0.072, 0.056
CNSHA 8 0.1232, 0.114, 0.102, 0.083, 0.074, 0.06
CNSHA 9 0.125, 0.116, 0.1085, 0.1, 0.083, 0.06
CNSHA 10 0.179, 0.133, 0.085, 0.06, 0.05
BUSAN 3 0.0986, 0.09, 0.0855, 0.07875, 0.07, 0.0435, 0.003
BUSAN 4 0.0966, 0.087, 0.057
BUSAN 5 0.1219, 0.11, 0.062
BUSAN 6 0.1435, 0.12375, 0.0875, 0.076
BUSAN 7 0.1425, 0.0935, 0.085, 0.0675
BUSAN 8 0.1611, 0.138, 0.1185, 0.0995, 0.09, 0.0838, 0.065
BUSAN 9 0.1659, 0.1538, 0.1465, 0.1435, 0.0935, 0.067, 0.052
BUSAN 10 0.1041, 0.0945, 0.0835, 0.075, 0.0285, 0.018, 0.015
BUSAN 11 0.1123, 0.0935, 0.087, 0.0155
BUSAN 12 0.1123, 0.0965, 0.0803
BUSAN 13 0.115, 0.1095, 0.10195, 0.09595, 0.0648, 0.022
BUSAN 14 0.1191, 0.101, 0.0975, 0.014
BUSAN 15 0.1211, 0.0995, 0.081, 0.0695, 0.015
KRINC 3 0.0685, 0.0418, 0.0188
KRINC 4 0.3215, 0.07185, 0.043
KRINC 5 0.386, 0.08861, 0.072
KRINC 6 0.248, 0.076468, 0.02085
KRINC 7 0.4375, 0.333, 0.2728
KRINC 8 0.4525, 0.40729, 0.38, 0.344
KRINC 9 0.51596, 0.491, 0.4585
"""

# eps 값을 이중리스트에 담기
port_minpts_eps = []

for line in eps_by_eyes.strip().split('\n'):
    parts=line.split()
    port=parts[0]
    minpts=int(parts[1])
    eps_values=[float(eps.strip(',')) for eps in parts[2:]]
    for eps in eps_values:
        port_minpts_eps.append([port, minpts, eps])

# port_minpts_eps

In [ ]:
port_minpts_eps = sorted(port_minpts_eps, key=lambda x: (x[1], x[2]))

In [ ]:
df[df['TO_PORT_CD'] == 'SGSIN'].shape

In [ ]:
def show_sil(save_all_info,size,port_name):
    lim = [i for i in range(len(save_all_info))]
    title = [f"{str(item['sample'])} + {str(item['eps'])}" for item in save_all_info]
    silhouette_scores = [item['sil'] for item in save_all_info]
    sil_under_0 = [item['sil_0'] for item in save_all_info]
    
    # 이중 y축 설정
    fig, ax1 = plt.subplots(figsize=(14, 8))
    
    color = 'tab:blue'
    ax1.set_xlabel('min_samples')
    ax1.set_ylabel('Silhouette Score', color=color)
    ax1.plot(lim, silhouette_scores, color=color, marker='o', label='Silhouette Score')
    ax1.tick_params(axis='y', labelcolor=color)
    ax1.set_xticks(lim,title,rotation=90)
    
    # y축 범위 설정
    #ax1.set_ylim(0, 1)  # Silhouette Score y축 범위 설정
    #ax1.set_yticks([0, 0.25, 0.5, 0.75, 1])
    #ax1.set_yticklabels(silhouette_scores)
    
    ax2 = ax1.twinx()  # 이중 y축 공유
    color = 'tab:red'
    ax2.set_ylabel('sil_under_0', color=color)
    ax2.plot(lim, sil_under_0, color=color, marker='x', linestyle='--', label='sil_under_0')
    ax2.tick_params(axis='y', labelcolor=color)
    ax2.set_xticks(lim,title,rotation=90)
    
    # y축 범위 설정
    # ax2.set_ylim(-0.1, 1.1)  # Silhouette Score y축 범위 설정
    # ax2.set_yticks(np.linspace(-0.1,1.1,len(set(sil_under_0))))
    # ax2.set_yticklabels(np.sort(list(set(sil_under_0))))
    
    fig.tight_layout()  # 그래프 간격 조정
    plt.title(f'Silhouette Score and sil_under_0 of {port_name}')
    fig.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    
    plt.grid()
    ax1.axhline(y=0.5, color='g', linestyle='--')  # y=0 인 곳에 빨간색 점선 추가

    
    ax2.axhline(y=int(size/10), color='g', linestyle='--')  # y=0 인 곳에 빨간색 점선 추가
    ax1.grid(axis='x', color='blue', linestyle=':', linewidth=0.5)
    

    for a,b,i in zip(silhouette_scores,sil_under_0,lim): 
        if a>=0.5 and b<=int(size/10): 
            ax1.plot(i,a,color='magenta',marker='o')
            ax2.plot(i,b,color='magenta',marker='o')
            ax1.plot(i,abs(a-b/100),color='black',marker='o')
            
            #print(a,b)
   # plt.savefig(f'Silhouette Score and sil_under_0 of {port_name}.png',format='png')
    
    plt.show()

In [ ]:
df = pd.read_csv('REAL_LAST.csv')

# 각 항구별 데이터프레임을 필터링하여 dictionary에 저장
ports = {
    'SGSIN': df[df['TO_PORT_CD'] == 'SGSIN'],
    # 'HKHKG': df[df['TO_PORT_CD'] == 'HKHKG']
    #'KRINC': df[df['TO_PORT_CD'] == 'KRINC'],
    #'KRPUS': df[df['TO_PORT_CD'] == 'KRPUS'],
    #'KRBNP': df[df['TO_PORT_CD'] == 'KRBNP'],
    #'CNSHA': df[df['TO_PORT_CD'] == 'CNSHA'],
    #'BUSAN': df[(df['TO_PORT_CD'] == 'KRPUS') | (df['TO_PORT_CD'] == 'KRBNP')]
}


# 각 항구별로 DBSCAN 군집화 수행
for port_name, port_data in ports.items():
    ad = port_data.copy()
    keep_info = []
    data = port_data[['WAIT_LAT', 'WAIT_LON']].values
    for minpts, eps in [(item[1], item[2]) for item in port_minpts_eps if item[0] == port_name]:
        # DBSCAN 군집화 수행
        labels = DBSCAN(eps=eps, min_samples=minpts).fit_predict(data)

        # 실루엣 샘플 평가 값 추가
        if len(set(labels)) > 1:
            sil_samples = silhouette_samples(data, labels)
            ad['label'] = labels
            ad['sil_samples'] = sil_samples
            temp = {} 
            temp['eps'] = eps
            temp['sample'] = minpts 
            temp['sil'] = silhouette_score(data,labels) 
            temp['sil_0'] = len(sil_samples[sil_samples < 0])

            keep_info.append(temp)
        else:
            sil_samples = [-1] * len(labels)  # 군집이 하나인 경우 실루엣 계수 계산 불가

        #port_data[f'sil_sample_{minpts}_{eps}'] = sil_samples
        #port_data[f'cluster_{minpts}_{eps}'] = labels

    show_sil(keep_info,port_data.shape[0],f"{port_name}_{port_data.shape[0]}")
    # with open(f"{port_name}_{port_data.shape[0]}.txt",'w') as f:
    #     for item in keep_info:
    #         f.write(f'{item}\n')
    print('='*100)
print('END')

# 군집 분석 (김묘경)

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
from datetime import timedelta

In [ ]:
df_scaled = pd.read_csv('keep.csv')
df_scaled.shape

In [ ]:
def str_to_timedelta(time_str):
    days, time = time_str.split(' days ')
    hours, minutes, seconds = map(int, time.split(':'))
    return timedelta(days=int(days), hours=hours, minutes=minutes, seconds=seconds)
df_scaled['WAITING_TIME(H:M:S)'] = df_scaled['WAITING_TIME(H:M:S)'].apply(str_to_timedelta)
#df_scaled['WAITING_TIME(H:M:S)'] = df_scaled['WAITING_TIME(H:M:S)'].apply(lambda x: x.total_seconds())

In [ ]:
# 데이터 분할
df_KRINC=df_scaled[df_scaled['TO_PORT_CD']=='KRINC']
df_KRBNP=df_scaled[df_scaled['TO_PORT_CD']=='KRBNP']
df_KRPUS=df_scaled[df_scaled['TO_PORT_CD']=='KRPUS']
df_CNSHA=df_scaled[df_scaled['TO_PORT_CD']=='CNSHA']
df_SGSIN=df_scaled[df_scaled['TO_PORT_CD']=='SGSIN']
df_HKHKG=df_scaled[df_scaled['TO_PORT_CD']=='HKHKG']
# 부산 통합
df_BUSAN=df_scaled[(df_scaled['TO_PORT_CD']=='KRBNP')|(df_scaled['TO_PORT_CD']=='KRPUS')]

# 추후 함수 반복 실행에 필요한 사전 생성
df_dic={
    'KRINC':df_KRINC,
    'CNSHA':df_CNSHA,
    'HKHKG':df_HKHKG,
    'KRBNP':df_KRBNP,
    'SGSIN':df_SGSIN,
    'KRPUS':df_KRPUS,
    'BUSAN':df_BUSAN,
}


# df_scaled['WAVE_avg(m)']=(df_scaled['START_WAVE(m)']+df_scaled['END_WAVE(m)'])/2
# df_scaled['WIND_avg(kn)']=(df_scaled['START_WIND(kn)']+df_scaled['END_WIND(kn)'])/2
# df_scaled['CURRENT_avg(kn)']=(df_scaled['START_CURRENT(kn)']+df_scaled['END_CURRENT(kn)'])/2
# df_scaled['DRAFT(m)']=df_scaled['START_DRAFT'] # draft: start와 end값이 동일함.
# df_scaled['HEIGHT_avg(m)']=(df_scaled['START_HEIGHT']+df_scaled['END_HEIGHT'])/2

# # 데이터 분할
# df_KRINC=df_scaled[df_scaled['TO_PORT_CD']=='KRINC']
# df_KRBNP=df_scaled[df_scaled['TO_PORT_CD']=='KRBNP']
# df_KRPUS=df_scaled[df_scaled['TO_PORT_CD']=='KRPUS']
# df_CNSHA=df_scaled[df_scaled['TO_PORT_CD']=='CNSHA']
# df_SGSIN=df_scaled[df_scaled['TO_PORT_CD']=='SGSIN']
# df_HKHKG=df_scaled[df_scaled['TO_PORT_CD']=='HKHKG']
# # 부산 통합
# df_BUSAN=df_scaled[(df_scaled['TO_PORT_CD']=='KRBNP')|(df_scaled['TO_PORT_CD']=='KRPUS')]

# # 추후 함수 반복 실행에 필요한 사전 생성
# df_dic={
#     'KRINC':df_KRINC,
#     'CNSHA':df_CNSHA,
#     'HKHKG':df_HKHKG,
#     'KRBNP':df_KRBNP,
#     'SGSIN':df_SGSIN,
#     'KRPUS':df_KRPUS,
#     'BUSAN':df_BUSAN,
# }

## 전체 군집에 대한 분석

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
from datetime import timedelta

In [ ]:
#df = pd.read_csv('THE_LAST_df.csv')
df = pd.read_csv('THE_LAST_df_0619.csv',index_col=0)
df.shape

In [ ]:
df['loc'] = ['sg' if i == 'SGSIN' else 'ea' for i in df['TO_PORT_CD']]
df

In [ ]:
df.rename(columns={'WAITING_TIME(H:M:S)':'WAITING_TIME'}, inplace=True)

In [ ]:
def str_to_timedelta(time_str):
    days, time = time_str.split(' days ')
    hours, minutes, seconds = map(int, time.split(':'))
    return timedelta(days=int(days), hours=hours, minutes=minutes, seconds=seconds)
df['WAITING_TIME'] = df['WAITING_TIME'].apply(str_to_timedelta)
#df_scaled['WAITING_TIME(H:M:S)'] = df_scaled['WAITING_TIME(H:M:S)'].apply(lambda x: x.total_seconds())

In [ ]:
v = df.copy()
v = v[['WAITING_TIME', #'START_WAVE(m)', 'START_WIND(kn)', 'START_CURRENT(kn)', 'START_HEIGHT', 'START_DRAFT', 'END_WAVE(m)', 'END_WIND(kn)',
#'END_CURRENT(kn)', 'END_HEIGHT', 
       #'START_SPEED', 'END_SPEED', 
       'GROSS_TONNAGE', 'Cluster_Label', 'SQUARE', 'WAVE_avg(m)', 'WIND_avg(kn)', 'CURRENT_avg(kn)', 'DRAFT(m)','HEIGHT_avg(m)','loc']]
for i in v.columns.to_list(): 
    if i == 'Cluster_Label' or i=='loc': continue
    #display(v[[i]].agg(['mean','std','max','min']))
    agg = v.groupby('Cluster_Label')[[i]].agg(['mean','std','max','min']).reset_index()
    #agg = v.groupby('loc')[[i]].agg(['mean','std','max','min'])
    agg.to_csv(f'0618_{i}_recsv.csv',index=False)
    display(agg)
print('='*100)

In [ ]:
v['WAITING_TIME'] = v['WAITING_TIME'].apply(lambda x: x.total_seconds()/3600) 
for i in v.columns.to_list(): 
    if i == 'Cluster_Label' or i == 'loc': continue
    #display(v.groupby('Cluster_Label')[[i]].agg(['mean','std','max','min']))

   # sns.set_style('whitegrid')
    #sns.boxplot(data=v, x='Cluster_Label',y=v[i])
    sns.boxplot(data=v, x='loc',y=v[i])
    plt.savefig(f'0618_{i}_re_plt.png',format='png')
    plt.show()
    print('='*100)

## (번외) 실제 사용된 선박들의 선박 정보 파악 

In [ ]:
import pandas as pd 
from datetime import timedelta
df = pd.read_csv('THE_LAST_df.csv')
df.shape

In [ ]:
def str_to_timedelta(time_str):
    days, time = time_str.split(' days ')
    hours, minutes, seconds = map(int, time.split(':'))
    return timedelta(days=int(days), hours=hours, minutes=minutes, seconds=seconds)
df['WAITING_TIME(H:M:S)'] = df['WAITING_TIME(H:M:S)'].apply(str_to_timedelta)
#df_scaled['WAITING_TIME(H:M:S)'] = df_scaled['WAITING_TIME(H:M:S)'].apply(lambda x: x.total_seconds())

In [ ]:
df[['Unnamed: 0','GROSS_TONNAGE','START_DRAFT']]
al = df[['GROSS_TONNAGE','START_DRAFT','WAITING_TIME(H:M:S)']].agg(['mean','std','max','min']) 
al.to_csv('0619_선박정보.csv',index=False)

In [ ]:
al

# 1개 선박에 대해 항해 분리 후 경로 시각화 

In [ ]:
import pandas as pd
import numpy as np 
import folium

In [ ]:
######## 선박 한대 만 시각화
######### 지도 시각화
def One_Mapping(df,locs): 
    m = folium.Map(location=[df['LAT'].mean(), df['LON'].mean()], zoom_start=10)

    # 좌표, 시간 정리 
    #loc = df[['LAT','LON']].values.tolist() 
    #time = np.datetime_as_string(df['VSL_TIMESTAMP'].values).tolist()
    # 좌표마다 그리기 
    #for l,t in zip(loc,time): 
        # 시간 적기
    #    title = f'{l} \n --- \n{time}'
    #    folium.Circle(
    #           l,popup=title,tooltip=title,radius=100
    #       ).add_to(m)
    #folium.PolyLine(locs,color='red').add_to(m)
    for trip, trip_df in df.groupby('TRIP'): 
        lat_lon = [(trip_df.iloc[i]['LAT'],trip_df.iloc[i]['LON']) for i in range(trip_df.shape[0])]      
        if trip == 0:   
            folium.PolyLine(lat_lon,color='red').add_to(m)
        elif trip == 1: 
            folium.PolyLine(lat_lon,color='gold').add_to(m)
        elif trip == 2:
            folium.PolyLine(lat_lon,color='yellow').add_to(m)
        elif trip == 3:
            folium.PolyLine(lat_lon,color='forestgreen').add_to(m)
        elif trip == 4:
            folium.PolyLine(lat_lon,color='blue').add_to(m)
        elif trip == 5:
            folium.PolyLine(lat_lon,color='purple').add_to(m)
        elif trip == 6:
            folium.PolyLine(lat_lon,color='white').add_to(m)
    return m

In [ ]:
before_cut = pd.read_csv('2.2_all_drop_outlier.csv')
before_cut.shape

In [ ]:
a['TRIP'].dtype

In [ ]:
a = before_cut[before_cut['VSL_ID']=='af90f964-a74f-3875-a75b-c06ee10503b4']
a['VSL_TIMESTAMP'] = pd.to_datetime(a['VSL_TIMESTAMP'],format='%Y-%m-%d %H:%M:%S')
a = temp_b.sort_values('VSL_TIMESTAMP')#.reset_index(drop=True)
lat_lon = []
for i in range(a.shape[0]): 
    lat_lon.append((a.iloc[i]['LAT'],a.iloc[i]['LON']))
m = One_Mapping(a,lat_lon)
m.save('0619_check_one_ship_afer_cut.html')
#m.save('0619_check_one_ship.html')
print('end')